In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import time 
import string
from progressbar import ProgressBar
pbar = ProgressBar()
from tqdm import tqdm_notebook as tqdm
import pickle

url = "https://www.sports-reference.com/cbb/players"

In [2]:
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

### Function to Parse urls

In [3]:
# Function to parse through the url
def getAndParseUrl(url):
    results = requests.get(url)
#     time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

### Remove "/cbb/players" from url

In [4]:
root_url = url[:32]
root_url

'https://www.sports-reference.com'

### Take all letters from the link

In [5]:
letter_link = []
for links in tqdm(string.ascii_lowercase):
    results = soup.find('a', {'href': f"/cbb/players/{links}-index.html"})['href']
    letter_link.append(results)

###  Add the root url to the ends of the letters link

In [6]:
full_url = []
for link in tqdm(letter_link):
    letters_url = root_url + link
    full_url.append(letters_url)

### Parse through about the letter url

In [7]:
parsed_letters = []
for urls in tqdm(full_url):
    soup = getAndParseUrl(urls)
#     print(soup)
    parsed_letters.append(soup)

### Function to Parse through each individual page and obtain player url and year played

In [8]:
def p_parsing(page_number):
    player_time = []
    player = parsed_letters[page_number].find_all('p')[1:]
    date = parsed_letters[page_number].find_all('p')[1:]
    for players, time in zip(player, date):
        try:
            played = players.a['href']
            times = time.small.text[1:10]
#             take only dates after 2001
            if int(times[:4]) >= 2001:
                player_time.append({"url":root_url +played, "start_year": int(times[:4]), "end_year": int(times[5:9])})
        except:
            pass
    return player_time

### Parse through url of each player

In [9]:
full_list = []
for i in tqdm(range(0,26)):
    full_list.extend(p_parsing(i))

### Take only the url from dictionary of full_list

In [10]:
players_site = []
for dic in tqdm(full_list):
    urls = dic['url']
    players_site.append(urls)

In [77]:
all_players = []
rest_of_players = []
for d in tqdm(full_list[16613:]):
    url = d['url']
    soup = getAndParseUrl(url)
    exp1 = soup
    rest_of_players.append(exp1)

### Function to get stats of each player

In [79]:
def stat_compiler(list_soups):
    list1 = []
    for exp in tqdm(list_soups):
        try:
            name = exp.h1.text
            height = exp.find('span', {'itemprop': "height"}).text
            weight = exp.find('span', {'itemprop': "weight"}).text[:3]
            position = exp.p.strong.nextSibling.strip()
            games = exp.select("tfoot td[data-stat ='g']")[0].text
            games_started = exp.select("tfoot td[data-stat= 'gs']")[0].text
            minutes_per = exp.select("tfoot td[data-stat= 'mp_per_g']")[0].text
            field_goal = exp.select("tfoot td[data-stat= 'fg_per_g']")[0].text
            field_attmp = exp.select("tfoot td[data-stat= 'fga_per_g']")[0].text
            field_pct = exp.select("tfoot td[data-stat= 'fg_pct']")[0].text
            two_ptrs = exp.select("tfoot td[data-stat= 'fg2_per_g']")[0].text
            two_pattmp = exp.select("tfoot td[data-stat= 'fg2a_per_g']")[0].text
            two_pct = exp.select("tfoot td[data-stat= 'fg2_pct']")[0].text
            three_ptrs = exp.select("tfoot td[data-stat= 'fg3_per_g']")[0].text
            three_pattmp = exp.select("tfoot td[data-stat= 'fg3a_per_g']")[0].text
            three_pct = exp.select("tfoot td[data-stat= 'fg_pct']")[0].text
            free_throws = exp.select("tfoot td[data-stat= 'ft_per_g']")[0].text
            free_attmp = exp.select("tfoot td[data-stat= 'fta_per_g']")[0].text
            free_pct = exp.select("tfoot td[data-stat= 'ft_pct']")[0].text
            tot_reb = exp.select("tfoot td[data-stat= 'trb_per_g']")[0].text
            assist = exp.select("tfoot td[data-stat= 'ast_per_g']")[0].text
            steals = exp.select("tfoot td[data-stat= 'stl_per_g']")[0].text
            blocks = exp.select("tfoot td[data-stat= 'blk_per_g']")[0].text
            points = exp.select("tfoot td[data-stat= 'pts_per_g']")[0].text
            oreb = exp.select("tfoot td[data-stat= 'orb_per_g']")[0].text
            dreb = exp.select("tfoot td[data-stat= 'drb_per_g']")[0].text
            tov = exp.select("tfoot td[data-stat= 'tov_per_g']")[0].text
            exp.select("tfoot  td[data-stat='tov_per_g']")[0].text
            position = position.strip('\n')

            dict1 = {'name': name,'height': height, 'weight': weight, 'position': position,
                    'games_played': games, 'games_started': games_started, 'min_per': minutes_per,
                    'field_goal': field_goal, 'field_attmps': field_attmp, 'field_pct': field_pct,
                    'two_pointer': two_ptrs, 'two_pattamps': two_pattmp, 'two_pct': two_pct,
                    'three_ptrs': three_ptrs, 'three_pattmp': three_pattmp, 'three_pct': three_pct,
                    'free_throws': free_throws, 'free_attmps': free_attmp, 'free_pct': free_pct,
                    'assists': assist, 'steals': steals, 'blocks': blocks, 
                    'points': points, 'off_reb': oreb, 'def_reb': dreb,'total_reb': tot_reb, 'turnovers': tov}
            list1.append(dict1)
        except:
            print('f could not get {position}')
    return list1

### Assign Function to a Variable

In [ ]:
full_list  = stat_compileriler(all_players)

In [88]:
new_list = stat_compiler(rest_of_players)

In [84]:
len(final_list) + len(new_list)

31860

In [86]:
players_stats = final_list + new_list

### Pickle all list of items

In [76]:
with open("final_list.pickle", "wb") as fp:
    pickle.dump(final_list, fp)

In [85]:
with open("new_list.pickle", "wb") as fp:
    pickle.dump(new_list, fp)

In [87]:
with open("players_stats.pickle", "wb") as fp:
    pickle.dump(players_stats, fp)

In [ ]:
# import json
# with open('all_players_16612.json', 'w') as f:
#     json.dump(all_players, f)

###  Turn into DataFrame and Save to CSV File

In [90]:
df = pd.DataFrame.from_dict(players_stats)

In [91]:
df= df[['name', 'height', 'weight', 'position', 'games_played', 'games_started', 'min_per', 'field_goal',
            'field_attmps', 'field_pct', 'two_pointer', 'two_pattamps', 'two_pct', 'three_ptrs', 'three_pattmp',
            'three_pct', 'free_throws', 'free_attmps', 'free_pct', 'assists', 'steals', 'blocks', 'points','off_reb',
            'def_reb', 'total_reb', 'turnovers']]

In [95]:
df.shape

(31860, 27)

In [96]:
df.to_csv(r'players_stats.csv')